Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [7]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic regression

In [5]:
# L2 regularization: Beta*(1/2) *norm(weights)[2 2] = Beta*(1/2)*(sum(weight**2)) = Beta*tf.nn.l2_loss(weights)
beta = 0.01
batch_size = 128
num_feature = image_size*image_size


graph = tf.Graph()

with graph.as_default():
    
    # Input
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_feature))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    #weight
    weights = tf.Variable(tf.truncated_normal([num_feature, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    
    #compute loss
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    #*******************IMPORTANT***********************
    regularizers = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels)) + beta*regularizers
    #*******************IMPORTANT***********************
    
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Run session, batch gradient descent (SGD)
num_step = 3001


with tf.Session(graph = graph) as session:
    #init variable
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_step):
        
        #choose offset for the next batch
        offset = (step*batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #feed the placeholder
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        #run
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 500) == 0:
            print('Minibatch loss at step %d: %f' %(step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('-----------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        
    



Initialized
Minibatch loss at step 0: 47.052750
Minibatch accuracy: 12.5%
Validation accuracy: 14.4%
Minibatch loss at step 500: 0.981215
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 1000: 0.713218
Minibatch accuracy: 84.4%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 0.561011
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 2000: 0.896417
Minibatch accuracy: 78.9%
Validation accuracy: 81.8%
Minibatch loss at step 2500: 0.651875
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.709218
Minibatch accuracy: 82.8%
Validation accuracy: 81.6%
-----------------
Test accuracy: 88.9%


### Neural network

In [7]:
# L2 regularization: Beta*(1/2) *norm(weights)[2 2] = Beta*(1/2)*(sum(weight**2)) = Beta*tf.nn.l2_loss(weights)
beta = 0.01
batch_size = 128
num_feature = image_size*image_size
num_hidden = 1024


graph = tf.Graph()

with graph.as_default():
    
    # Input
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_feature))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    #weight
    weights_1 = tf.Variable(tf.truncated_normal([num_feature, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    hidden = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden = tf.nn.relu(hidden)
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    
    #compute loss
    logits = tf.matmul(hidden, weights_2) + biases_2
    
    #*******************IMPORTANT***********************
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels)) + beta*regularizers
    #*******************IMPORTANT***********************
    
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2)+ biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2)+ biases_2)

In [8]:
# Run session, batch gradient descent (SGD)
num_step = 3001


with tf.Session(graph = graph) as session:
    #init variable
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_step):
        
        #choose offset for the next batch
        offset = (step*batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #feed the placeholder
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        #run
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 500) == 0:
            print('Minibatch loss at step %d: %f' %(step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('-----------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        
    



Initialized
Minibatch loss at step 0: 3520.919189
Minibatch accuracy: 20.3%
Validation accuracy: 24.6%
Minibatch loss at step 500: 21.450735
Minibatch accuracy: 81.2%
Validation accuracy: 85.0%
Minibatch loss at step 1000: 0.849083
Minibatch accuracy: 86.7%
Validation accuracy: 82.9%
Minibatch loss at step 1500: 0.591086
Minibatch accuracy: 87.5%
Validation accuracy: 83.6%
Minibatch loss at step 2000: 0.782870
Minibatch accuracy: 80.5%
Validation accuracy: 83.2%
Minibatch loss at step 2500: 0.616896
Minibatch accuracy: 85.9%
Validation accuracy: 84.0%
Minibatch loss at step 3000: 0.673386
Minibatch accuracy: 85.9%
Validation accuracy: 84.1%
-----------------
Test accuracy: 90.9%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

my comment: minibatch get 100% accuracy but lower in test set

---

In [14]:
beta = 0.001
batch_size = 128
num_feature = image_size*image_size
num_hidden = 1024


graph = tf.Graph()

with graph.as_default():
    
    # Input
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_feature))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    #weight
    weights_1 = tf.Variable(tf.truncated_normal([num_feature, num_hidden]))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    hidden = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden = tf.nn.relu(hidden)
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    
    #compute loss
    logits = tf.matmul(hidden, weights_2) + biases_2
    
    #*******************IMPORTANT***********************
    #regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels)) #+ beta*regularizers
    #*******************IMPORTANT***********************
    
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2)+ biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2)+ biases_2)
    

    
# Run session, batch gradient descent (SGD)
num_step = 101 #important
num_batches = 3 #important

with tf.Session(graph = graph) as session:
    #init variable
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_step):
        
        #choose offset for the next batch
        
        #important
        offset = ((step % num_batches)*batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #feed the placeholder
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        
        #run
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 2) == 0:
            print('Minibatch loss at step %d: %f' %(step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('-----------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
        


Initialized
Minibatch loss at step 0: 390.691711
Minibatch accuracy: 9.4%
Validation accuracy: 27.4%
Minibatch loss at step 2: 1506.278076
Minibatch accuracy: 32.0%
Validation accuracy: 33.2%
Minibatch loss at step 4: 534.505127
Minibatch accuracy: 44.5%
Validation accuracy: 51.9%
Minibatch loss at step 6: 201.383591
Minibatch accuracy: 68.8%
Validation accuracy: 67.8%
Minibatch loss at step 8: 59.164742
Minibatch accuracy: 86.7%
Validation accuracy: 68.8%
Minibatch loss at step 10: 111.061172
Minibatch accuracy: 86.7%
Validation accuracy: 70.5%
Minibatch loss at step 12: 10.060159
Minibatch accuracy: 92.2%
Validation accuracy: 74.1%
Minibatch loss at step 14: 3.400304
Minibatch accuracy: 95.3%
Validation accuracy: 73.8%
Minibatch loss at step 16: 0.105126
Minibatch accuracy: 99.2%
Validation accuracy: 74.0%
Minibatch loss at step 18: 0.292661
Minibatch accuracy: 99.2%
Validation accuracy: 74.1%
Minibatch loss at step 20: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.1%
M

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [29]:
#Some different
#+ Dropout, keep_prob = 0.5
#+ stddev = 0.1 for weights_1, weights_2
#+ Number of step: 10 001

beta = 0.0001
batch_size = 128
num_feature = image_size*image_size
num_hidden = 1024


graph = tf.Graph()

with graph.as_default():
    
    # Input
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_feature))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    
    #weight
    weights_1 = tf.Variable(tf.truncated_normal([num_feature, num_hidden], stddev=0.1))
    biases_1 = tf.Variable(tf.zeros([num_hidden]))
    
    hidden = tf.matmul(tf_train_dataset, weights_1) + biases_1
    #add relu
    hidden = tf.nn.relu(hidden)
    
    #dropout
    keep_prob = tf.placeholder(dtype = tf.float32)
    hidden = tf.nn.dropout(hidden, keep_prob = keep_prob)
    
    weights_2 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    
    #compute loss
    logits = tf.matmul(hidden, weights_2) + biases_2
    
    #*******************IMPORTANT***********************
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels)) + beta*regularizers

    #*******************IMPORTANT***********************
    
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #predict
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2)+ biases_2)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2)+ biases_2)

In [30]:
# Run session, batch gradient descent (SGD)
num_step = 10001


with tf.Session(graph = graph) as session:
    #init variable
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_step):
        
        #choose offset for the next batch
        offset = (step*batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #feed the placeholder
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob:0.5}
        
        #run
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 500) == 0:
            print('Minibatch loss at step %d: %f' %(step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('-----------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.313994
Minibatch accuracy: 7.8%
Validation accuracy: 27.4%
Minibatch loss at step 500: 1.022395
Minibatch accuracy: 81.2%
Validation accuracy: 84.8%
Minibatch loss at step 1000: 0.922510
Minibatch accuracy: 82.8%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 0.631050
Minibatch accuracy: 89.8%
Validation accuracy: 85.8%
Minibatch loss at step 2000: 0.908876
Minibatch accuracy: 79.7%
Validation accuracy: 86.2%
Minibatch loss at step 2500: 0.601869
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.708484
Minibatch accuracy: 87.5%
Validation accuracy: 87.8%
Minibatch loss at step 3500: 0.748642
Minibatch accuracy: 85.2%
Validation accuracy: 87.8%
Minibatch loss at step 4000: 0.751345
Minibatch accuracy: 82.8%
Validation accuracy: 88.5%
Minibatch loss at step 4500: 0.785790
Minibatch accuracy: 81.2%
Validation accuracy: 88.8%
Minibatch loss at step 5000: 0.554996
Minibatch accuracy: 87.5%
Validation accuracy

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


#### improvements:
+ Learning rate decay
+ stddev:0.1
+ 3 hidden layer
+ dropout
+ L2 regularization
+ Increase num step to 30001

In [51]:
#ref: https://github.com/vanhuyz/udacity-dl/blob/master/3_regularization.ipynb

def prediction(tf_dataset, W1, b1, W2, b2, W3, b3, W4, b4):
    Y1 = tf.matmul(tf_dataset, W1) + b1
    
    X2 = tf.nn.relu(Y1)
    Y2 = tf.matmul(X2, W2) + b2
    
    X3 = tf.nn.relu(Y2)
    Y3 = tf.matmul(X3, W3) + b3
    
    X4 = tf.nn.relu(Y3)
    Y4 = tf.matmul(X4, W4) + b4
    
    return tf.nn.softmax(Y4)

##### Init graph

In [56]:
num_hidden_1 = 1024
num_hidden_2 = 512
num_hidden_3 = 64

num_feature = image_size*image_size
batch_size = 128

beta = 5e-4


graph = tf.Graph()

with graph.as_default():
    
    #place holder
    tf_train_dataset = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_feature))
    tf_train_labels = tf.placeholder(dtype = tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #Variable, weight
    
    W1 = tf.Variable(tf.truncated_normal([num_feature, num_hidden_1], stddev = 0.1))
    W2 = tf.Variable(tf.truncated_normal([num_hidden_1, num_hidden_2], stddev = 0.1))
    W3 = tf.Variable(tf.truncated_normal([num_hidden_2, num_hidden_3], stddev = 0.1))
    W4 = tf.Variable(tf.truncated_normal([num_hidden_3, num_labels], stddev = 0.1))
    
    b1 = tf.zeros([num_hidden_1])
    b2 = tf.zeros([num_hidden_2])
    b3 = tf.zeros([num_hidden_3])
    b4 = tf.zeros([num_labels])
    
    keep_prob = tf.placeholder(dtype = tf.float32)
    
    #compute hidden_1
    hidden_1 = tf.matmul(tf_train_dataset, W1) + b1
    hidden_1 = tf.nn.relu(hidden_1)
    hidden_1 = tf.nn.dropout(hidden_1, keep_prob = keep_prob)
    
    #compute hidden_2
    hidden_2 = tf.matmul(hidden_1, W2) + b2
    hidden_2 = tf.nn.relu(hidden_2)
    hidden_2 = tf.nn.dropout(hidden_2, keep_prob = keep_prob)
    
    #compute hidden_3
    hidden_3 = tf.matmul(hidden_2, W3) + b3
    hidden_3 = tf.nn.relu(hidden_3)
    hidden_3 = tf.nn.dropout(hidden_3, keep_prob = keep_prob)
    
    #compute logits
    logits = tf.matmul(hidden_3, W4) + b4
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = tf_train_labels))
    
    #regularizer
    regularizers = tf.nn.l2_loss(W1) + tf.nn.l2_loss(W2) + tf.nn.l2_loss(W3) + tf.nn.l2_loss(W4)
    loss += beta * regularizers
    
    #optimizer
    global_step = tf.Variable(0)
    starter_learning_rate = 0.5
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,500,0.96)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    #predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = prediction(tf_valid_dataset, W1, b1, W2, b2, W3, b3, W4, b4)
    test_prediction = prediction(tf_test_dataset, W1,b1, W2, b2, W3, b3, W4, b4)
    

##### Run session

In [57]:
# Run session, batch gradient descent (SGD)
num_step = 30001


with tf.Session(graph = graph) as session:
    #init variable
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_step):
        
        #choose offset for the next batch
        offset = (step*batch_size) % (train_dataset.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        #feed the placeholder
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels, keep_prob:0.5}
        
        #run
        _,l,predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        
        if (step % 500) == 0:
            print('Minibatch loss at step %d: %f' %(step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
            
    print('-----------------')
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 9.079694
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 4.099854
Minibatch accuracy: 28.9%
Validation accuracy: 34.9%
Minibatch loss at step 1000: 2.725598
Minibatch accuracy: 69.5%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 1.976642
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 1.910960
Minibatch accuracy: 78.9%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 1.477919
Minibatch accuracy: 85.2%
Validation accuracy: 85.8%
Minibatch loss at step 3000: 1.367047
Minibatch accuracy: 85.9%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 1.331229
Minibatch accuracy: 78.9%
Validation accuracy: 86.5%
Minibatch loss at step 4000: 1.348836
Minibatch accuracy: 79.7%
Validation accuracy: 86.6%
Minibatch loss at step 4500: 1.158938
Minibatch accuracy: 80.5%
Validation accuracy: 87.1%
Minibatch loss at step 5000: 0.863952
Minibatch accuracy: 86.7%
Validation accurac